In [138]:
import numpy as np
import pandas as pd
from datetime import datetime

# Sample Trajectories

In [139]:
# Define Trajectory 1
passenger = [
    [40.747776, -73.987974, "2024-04-09T11:00:00Z"],
    [40.748776, -73.986974, "2024-04-09T11:05:00Z"],
    [40.749776, -73.985974, "2024-04-09T11:10:00Z"],
    [40.750776, -73.984974, "2024-04-09T11:15:00Z"],
]

drivers = [
    [
        [40.747876, -73.988074, "2024-04-09T11:00:00Z"],
        [40.748876, -73.987074, "2024-04-09T11:05:00Z"],
        [40.749876, -73.986074, "2024-04-09T11:10:00Z"],
        [40.750876, -73.985074, "2024-04-09T11:15:00Z"],
    ],
    [
        [40.747976, -73.988174, "2024-04-09T11:00:00Z"],
        [40.748976, -73.987174, "2024-04-09T11:05:00Z"],
        [40.749976, -73.986174, "2024-04-09T11:10:00Z"],
        [40.750976, -73.985174, "2024-04-09T11:15:00Z"],
    ],
]

driver1 = [
    [40.747876, -73.988074, "2024-04-09T11:00:00Z"],
    [40.748876, -73.987074, "2024-04-09T11:05:00Z"],
    [40.749876, -73.986074, "2024-04-09T11:10:00Z"],
    [40.750876, -73.985074, "2024-04-09T11:15:00Z"],
]

driver2 = [
    [40.747976, -73.988174, "2024-04-09T11:00:00Z"],
    [40.748976, -73.987174, "2024-04-09T11:05:00Z"],
    [40.749976, -73.986174, "2024-04-09T11:10:00Z"],
    [40.750976, -73.985174, "2024-04-09T11:15:00Z"],
]

### Data Conversion

In [140]:
def convert_to_unix(timestamp) -> float:
    datetime_obj = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
    unix_time = datetime_obj.timestamp()
    return unix_time

In [141]:
def convert_data(matrix):
    # Convert the timestamps to Unix time using pandas (you could also use datetime module in Python)
    timestamps = [convert_to_unix(date) for _ , _ , date in matrix]

    # Create a numpy array for latitudes, longitudes, and Unix timestamps
    latitudes = np.array([lat for lat, _, _ in matrix])
    longitudes = np.array([lng for _, lng, _ in matrix])
    timestamps = np.array(timestamps)

    return np.array(list(zip(latitudes, longitudes, timestamps)))


In [142]:
## Example Conversion
convert_data(passenger)

array([[ 4.0747776e+01, -7.3987974e+01,  1.7126604e+09],
       [ 4.0748776e+01, -7.3986974e+01,  1.7126607e+09],
       [ 4.0749776e+01, -7.3985974e+01,  1.7126610e+09],
       [ 4.0750776e+01, -7.3984974e+01,  1.7126613e+09]])

## Testing of Dynamic Time Warping Libraries

### TSLearn

In [143]:
from tslearn.metrics import dtw

# Numpy Matrix
ts1 = np.random.random((100,3))
ts2 = np.random.random((150,3))

# TSLean DTW test
distance = dtw(ts1, ts2)
distance


6.0401025336665315

In [144]:
# trajectory data to np
passenger_data = convert_data(passenger)
driver_data_1 = convert_data(driver1)
driver_data_2 = convert_data(driver2)

In [145]:
dist1 = dtw(passenger_data, driver_data_1)
dist2 = dtw(passenger_data, driver_data_2)

In [147]:
print(f"Distance from passenger to driver 1: {dist1}")
print(f"Distance from passenger to driver 2: {dist2}")

Distance from passenger to driver 1: 0.0002828427124789841
Distance from passenger to driver 2: 0.0005656854249529439
